In [2]:
from imagetomrz import ExtractData
from parsemrz import parse
list = ExtractData("C:\\Users\\iyedn\\OneDrive\\Desktop\\stage\\mrz-detection-master\\img\\210316052514-iceland-gender-neutral-passport.jpg")
l = parse(list)
print(l)

['THURIDUR OESP', 'AEVARSDOTTIR', 'P', 'ISL', 'A3536444', 'ISL', '121212', 'unspecified', '310310', '121212<1239', '7', '3', '8', '6', '8']
